# Cell 0: Tool Definition
from confluence_rag_integration import query_customer
from langchain.tools import tool
from typing import List, Optional
import json

@tool
def retrieve_knowledge(query: str) -> str:
    """
    Search and retrieve relevant documentation from Confluence knowledge base.
    Use this when you need to find information about products, procedures, 
    troubleshooting steps, or any documented knowledge to help answer questions.
    
    Args:
        query: The search query or question to find relevant documents
    Returns:
        A formatted string containing the retrieved documents
    """
    # This will be called by the tool_node which will provide customer_id and top_k
    # from the RunnableConfiguration - we'll handle that in the tool_node
    return "Tool execution handled by tool_node"

In [86]:
# Cell 1: Agent State Definition
from typing import TypedDict, Annotated, List, Optional
import operator
from langchain_core.messages import BaseMessage
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from dotenv import load_dotenv

load_dotenv()



True

In [87]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from typing import List
from confluence_rag_integration import query_customer
from langchain_google_genai import ChatGoogleGenerativeAI

@tool
def retrieve_knowledge(query: Annotated[str, "The search query or question to find relevant documents"], config: RunnableConfig) -> List[str]:
    """
    Search and retrieve relevant documentation from Confluence knowledge base.
    Use this when you need to find information about products, procedures, 
    troubleshooting steps, or any documented knowledge to help resolve tickets.
    
    Args:
        query: The search query or question to find relevant documents
    Returns:
        A string of the retrieved knowledge.
    """
    # configurable = config.get("configurable", {})
    # customer_id = configurable.get("customer_id", "acme_corp")
    # top_k = configurable.get("top_k", 3)

    result = query_customer(config["configurable"]["customer_id"], query, config["configurable"]["top_k"])
    formatted_results = []
    for doc in result.documents:
        source = doc.get("source", "Unknown")
        content = doc.get("content", "")
        formatted_results.append(f"Source: {source}:\n{content}")
    return formatted_results


In [88]:
# Cell 3: Define Graph Nodes
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # This should work!
    temperature=0
)

llm_with_tools = llm.bind_tools([retrieve_knowledge])
from langchain_core.messages import ToolMessage
from confluence_rag_integration import query_customer

def agent_node(state: MessagesState, config: RunnableConfig) -> dict:
    """Agent node that decides whether to search or respond."""
    messages = state["messages"]
    
    # Add system message for better agent behavior
    system_prompt = """You are a helpful support agent with access to a knowledge retrieval tool.
    Use the retrieve_knowledge tool to search for relevant documentation when answering questions.
    You can call the tool multiple times if needed to gather comprehensive information.
    Always search for information before providing an answer unless the question is clearly conversational."""
    
    # Invoke LLM with tools
    response = llm_with_tools.invoke([
        {"role": "system", "content": system_prompt},
        *messages
    ])
    
    return {"messages": [response]}

def tool_node(state: MessagesState, config: RunnableConfig) -> dict:
    """Execute tool calls from the last message."""
    last_message = state["messages"][-1]
    tool_messages = []
    
    # Get customer_id and top_k from config
    # configurable = config.get("configurable", {})
    # customer_id = configurable.get("customer_id", "acme_corp")
    # top_k = configurable.get("top_k", 3)
    
    # Execute each tool call
    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        
        # Execute the tool with config values
        if tool_name == "retrieve_knowledge":
            # Call the actual query_customer function with config values
            tool_result = retrieve_knowledge.invoke(tool_args, config)
            
            # Format results
            
            formatted_result = "\n\n---\n\n".join(tool_result)
        else:
            tool_result = f"Error: Unknown tool {tool_name}"
        
        # Create tool message
        tool_message = ToolMessage(
            content=formatted_result,
            tool_call_id=tool_call["id"],
            name=tool_name
        )
        tool_messages.append(tool_message)
    
    return {"messages": tool_messages}

In [89]:

from langgraph.graph import END
from typing import Literal


def should_continue(state: MessagesState) -> Literal["environment", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "Action"
    # Otherwise, we stop (reply to the user)
    return END


In [90]:
from langgraph.graph import StateGraph
from langgraph.graph.message import MessagesState

agent_builder = StateGraph(MessagesState)
agent_builder.add_node("agent_node", agent_node)
agent_builder.add_node("tool_node", tool_node)
agent_builder.set_entry_point("agent_node")
agent_builder.add_conditional_edges("agent_node", should_continue, {
    "Action": "tool_node",
    END: END
})
agent_builder.add_edge("tool_node", "agent_node")

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
app =agent_builder.compile(checkpointer=memory)





In [91]:
# Cell 6: Usage Examples
from langchain_core.messages import HumanMessage

# Example 1: Single query with custom config
print("=== Example 1: Password Reset Query ===")
result = app.invoke({
    "messages": [HumanMessage(content="How do I reset my password?")]
}, {
    "configurable": {
        "thread_id": "example-1",
        "customer_id": "acme_corp",
        "top_k": 3
    }
})

# Print the final response
print("\nAgent Response:")
result["messages"][-1].pretty_print()

# Show tool calls made
print("\nTool calls made:")
for msg in result["messages"]:
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for call in msg.tool_calls:
            print(f"- {call['name']}: {call['args']['query']}")

=== Example 1: Password Reset Query ===

Agent Response:
================================== Ai Message ==================================

To reset your password, please follow the steps relevant to your account type:

**For Business Systems (Single Sign-On) Password Reset:**

1.  Go to [iam.ucsd.edu/ResetPassword](https://iam.ucsd.edu/ResetPassword/).
2.  Enter your User ID (either your full UCSD Email address like `username@ucsd.edu` or your 6-character mainframe ID).
3.  Click **Continue**.
4.  Validate your identity by entering your Employee ID number (without leading zeros), the last 4 digits of your Social Security number, and your Date of Birth (mm/dd/yyyy format).
5.  Click **Continue**.
6.  Check your UCSD email for a temporary password. Your Department Security Administrator (DSA) will also be notified.
7.  Go to [iam.ucsd.edu/ResetPassword/login](https://iam.ucsd.edu/ResetPassword/login) and log in with the temporary password.
8.  You will be prompted to create a new, perman

In [92]:
# Cell 7: Example with Different Top K
from langchain_core.messages import HumanMessage

# Example 2: Query with more results
print("=== Example 2: Authentication Methods Query (top_k=5) ===")
result = app.invoke({
    "messages": [HumanMessage(content="What authentication methods do you support?")]
}, {
    "configurable": {
        "thread_id": "support-456", 
        "customer_id": "acme_corp",
        "top_k": 5  # Get more results
    }
})

# Print the final response
print("\nAgent Response:")
result["messages"][-1].pretty_print()

# Show tool calls made
print("\nTool calls made:")
for msg in result["messages"]:
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for call in msg.tool_calls:
            print(f"- {call['name']}: {call['args']['query']}")

=== Example 2: Authentication Methods Query (top_k=5) ===

Agent Response:
================================== Ai Message ==================================

We support two primary authentication methods:

1.  **Two-Step Login (Two-Factor Authentication)**: This involves using Duo for authentication. You can register multiple devices (mobile phones, tablets) and use a Duo hardware token.
2.  **Single Sign-On (SSO)**: This is used for Business Systems accounts.

Tool calls made:
- retrieve_knowledge: supported authentication methods


In [96]:
# Cell 8: Conversation with History
print("=== Example 3: Multi-turn Conversation ===")

# Use config with thread_id - MemorySaver handles conversation history
config = {
    "configurable": {
        "thread_id": "support-456",
        "customer_id": "acme_corp",
        "top_k": 3
    }
}

# Simulate a conversation
queries = [
    "What authentication methods do you support?",
    "Can you tell me more about SSO setup?",
    "What are the common SAML errors?"
]

for query in queries:
    print(f"\n👤 User: {query}")
    
    # Invoke with just the new message - MemorySaver handles history
    result = app.invoke({
        "messages": [HumanMessage(content=query)]
    }, config)
    
    # Print agent response
    print("\n🤖 Agent:")
    # Find the last AI message (not a tool call)
    for msg in reversed(result["messages"]):
        print(msg.content[:500] + "..." if len(msg.content) > 500 else msg.content)
            

=== Example 3: Multi-turn Conversation ===

👤 User: What authentication methods do you support?

🤖 Agent:
We support two primary authentication methods:

1.  **Two-Step Login (Two-Factor Authentication)**: This involves using Duo for authentication. You can register multiple devices (mobile phones, tablets) and use a Duo hardware token.
2.  **Single Sign-On (SSO)**: This is used for Business Systems accounts.
Source: data/customers/acme_corp/exports/Demo service project/Demo service project/Access & Security/Security/Two-Step Login/Add Another Device for Two-Step Login.md:
[Access & Security](../../../Access%20&%20Security.md) > [Security](../../Security.md) > [Two-Step Login](../Two-Step%20Login.md)  
# Add Another Device for Two-Step Login  
## Overview  
---  
Learn how to register a second (or more) device for your two-step login process. The interface will walk you through the process. You can r...

What authentication methods do you support?

How do I reset my password?
Common SA

In [94]:
# Cell 9: Analyze Agent Behavior
print("=== Agent Behavior Analysis ===")

# Test different query types
test_queries = [
    ("Hello!", "acme_corp", 3),  # Greeting - should not trigger search
    ("How do I configure LDAP?", "acme_corp", 3),  # Technical - should search
    ("What's the weather today?", "acme_corp", 3),  # Off-topic - might not search
    ("I'm having login issues with error code 403", "acme_corp", 5)  # Specific error - should search with more results
]

for i, (query, customer_id, top_k) in enumerate(test_queries):
    print(f"\nQuery: '{query}' (customer: {customer_id}, top_k: {top_k})")
    
    # Each test gets its own thread to avoid state pollution
    result = app.invoke({
        "messages": [HumanMessage(content=query)]
    }, {
        "configurable": {
            "thread_id": f"test-{i}",
            "customer_id": customer_id,
            "top_k": top_k
        }
    })
    
    # Count tool calls
    tool_calls = 0
    for msg in result["messages"]:
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            tool_calls += len(msg.tool_calls)
    
    print(f"Tool calls made: {tool_calls}")
    
    # Show brief response
    final_msg = result["messages"][-1]
    brief_response = final_msg.content[:100] + "..." if len(final_msg.content) > 100 else final_msg.content
    print(f"Response preview: {brief_response}")

=== Agent Behavior Analysis ===

Query: 'Hello!' (customer: acme_corp, top_k: 3)
Tool calls made: 0
Response preview: Hello! How can I help you today?

Query: 'How do I configure LDAP?' (customer: acme_corp, top_k: 3)
Tool calls made: 1
Response preview: I couldn't find any specific documentation on configuring LDAP. The search results provided informat...

Query: 'What's the weather today?' (customer: acme_corp, top_k: 3)
Tool calls made: 0
Response preview: I cannot provide real-time weather updates. My purpose is to search and retrieve information from a ...

Query: 'I'm having login issues with error code 403' (customer: acme_corp, top_k: 5)
Tool calls made: 1
Response preview: A 403 error typically indicates that your Business Systems account may be locked or revoked.

Here's...


In [ ]:
# Cell 8: Conversation with History
print("=== Example 3: Multi-turn Conversation ===")

# Use config with thread_id - MemorySaver handles conversation history
config = {"configurable": {"thread_id": "support-456"}}

# Simulate a conversation
queries = [
    "What authentication methods do you support?",
    "Can you tell me more about SSO setup?",
    "What are the common SAML errors?"
]

for query in queries:
    print(f"\n👤 User: {query}")
    
    # Invoke with just the new message - MemorySaver handles history
    result = app.invoke({
        "messages": [HumanMessage(content=query)],
        "customer_id": "acme_corp"
    }, config)
    
    # Print agent response
    print("\n🤖 Agent:")
    # Find the last AI message (not a tool call)
    for msg in reversed(result["messages"]):
        if msg.type == "ai" and not hasattr(msg, "tool_calls"):
            print(msg.content[:500] + "..." if len(msg.content) > 500 else msg.content)
            break

=== Example 3: Multi-turn Conversation ===

👤 User: What authentication methods do you support?

🤖 Agent:

👤 User: Can you tell me more about SSO setup?

🤖 Agent:

👤 User: What are the common SAML errors?

🤖 Agent:


In [ ]:
# Cell 9: Analyze Agent Behavior
print("=== Agent Behavior Analysis ===")

# Test different query types
test_queries = [
    "Hello!",  # Greeting - should not trigger search
    "How do I configure LDAP?",  # Technical - should search
    "What's the weather today?",  # Off-topic - might not search
    "I'm having login issues with error code 403"  # Specific error - should search
]

for i, query in enumerate(test_queries):
    print(f"\nQuery: '{query}'")
    
    # Each test gets its own thread to avoid state pollution
    result = app.invoke({
        "messages": [HumanMessage(content=query)],
        "customer_id": "acme_corp"
    }, {"configurable": {"thread_id": f"test-{i}"}})
    
    # Count tool calls
    tool_calls = 0
    for msg in result["messages"]:
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            tool_calls += len(msg.tool_calls)
    
    print(f"Tool calls made: {tool_calls}")
    
    # Show brief response
    final_msg = result["messages"][-1]
    brief_response = final_msg.content[:100] + "..." if len(final_msg.content) > 100 else final_msg.content
    print(f"Response preview: {brief_response}")

=== Agent Behavior Analysis ===

Query: 'Hello!'
Tool calls made: 0
Response preview: Hello there! How can I assist you today?

Query: 'How do I configure LDAP?'
Tool calls made: 0
Response preview: I need more information to help you configure LDAP.  LDAP configuration is highly specific to the op...

Query: 'What's the weather today?'
Tool calls made: 0
Response preview: I cannot provide you with the current weather. I do not have access to real-time information, such a...

Query: 'I'm having login issues with error code 403'
Tool calls made: 0
Response preview: I need more information to help you troubleshoot your login issue with error code 403.  The error co...


In [41]:
# Cell 10: Memory Inspection
print("=== Memory Inspection ===")

# Show conversation history for a thread
thread_id = "support-456"
state = app.get_state({"configurable": {"thread_id": thread_id}})

if state.values:
    print(f"\nConversation history for thread '{thread_id}':")
    messages = state.values.get("messages", [])
    
    for i, msg in enumerate(messages):
        if msg.type == "human":
            print(f"\n{i+1}. 👤 User: {msg.content}")
        elif msg.type == "ai" and not hasattr(msg, "tool_calls"):
            preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
            print(f"   🤖 Agent: {preview}")
        elif msg.type == "ai" and hasattr(msg, "tool_calls"):
            print(f"   🔍 Agent searched for: {', '.join([call['args']['query'] for call in msg.tool_calls])}")
        elif msg.type == "tool":
            print(f"   📚 Retrieved {len(msg.content)} characters of documentation")
    
    print(f"\nTotal messages in thread: {len(messages)}")
else:
    print(f"\nNo conversation history found for thread '{thread_id}'")

=== Memory Inspection ===

Conversation history for thread 'support-456':

1. 👤 User: What authentication methods do you support?
   🔍 Agent searched for: 

3. 👤 User: What authentication methods do you support?
   🔍 Agent searched for: 

5. 👤 User: Can you tell me more about SSO setup?
   🔍 Agent searched for: 

7. 👤 User: What are the common SAML errors?
   🔍 Agent searched for: 

9. 👤 User: How do I reset my password?
   🔍 Agent searched for: 

11. 👤 User: What authentication methods do you support?
   🔍 Agent searched for: 

13. 👤 User: Can you tell me more about SSO setup?
   🔍 Agent searched for: 

15. 👤 User: What are the common SAML errors?
   🔍 Agent searched for: 

Total messages in thread: 16


In [42]:
# Cell 9: Analyze Agent Behavior
print("=== Agent Behavior Analysis ===")

# Test different query types
test_queries = [
    "Hello!",  # Greeting - should not trigger search
    "How do I configure LDAP?",  # Technical - should search
    "What's the weather today?",  # Off-topic - might not search
    "I'm having login issues with error code 403"  # Specific error - should search
]

for query in test_queries:
    print(f"\nQuery: '{query}'")
    
    result = app.invoke({
        "messages": [HumanMessage(content=query)],
        "customer_id": "acme_corp"
    }, {"configurable": {"thread_id": "example-1"}})
    
    # Count tool calls
    tool_calls = 0
    for msg in result["messages"]:
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            tool_calls += len(msg.tool_calls)
    
    print(f"Tool calls made: {tool_calls}")
    
    # Show brief response
    final_msg = result["messages"][-1]
    brief_response = final_msg.content[:100] + "..." if len(final_msg.content) > 100 else final_msg.content
    print(f"Response preview: {brief_response}")

=== Agent Behavior Analysis ===

Query: 'Hello!'
Tool calls made: 0
Response preview: Hello there! How can I help you today?

Query: 'How do I configure LDAP?'
Tool calls made: 0
Response preview: Configuring LDAP is a complex process that depends heavily on your specific LDAP server (e.g., OpenL...

Query: 'What's the weather today?'
Tool calls made: 0
Response preview: I do not have access to real-time information, including weather. To get the weather for your locati...

Query: 'I'm having login issues with error code 403'
Tool calls made: 0
Response preview: A 403 error code, or "Forbidden" error, means the server understands your request but refuses to ful...
